In [ ]:
pip install pygraphviz

In [ ]:
import sempy.fabric as fabric  # Import the SemPy Fabric module for working with Fabric data
import pandas as pd  # Import Pandas for data manipulation and analysis
import networkx as nx  # Import NetworkX for creating and manipulating complex networks
from networkx.drawing.nx_agraph import to_agraph  # Import the function to convert NetworkX graphs to AGraph for Graphviz
from IPython.display import Image, display  # Import functions to display images and provide display functionalities in Jupyter notebooks

In [ ]:
workspace = 'Sneaker Workspace'
dataset = 'Sales & Returns Sample v201912'
measure = 'WIF Adjusted Sales'

In [ ]:
#Using fabric to return measures into pandas data frame
df = pd.DataFrame(fabric.list_measures(workspace=workspace, dataset=dataset))
G = nx.DiGraph()  # Create a directed graph

In [ ]:
# Function to parse formulas and extract dependencies for specific measure
def extract_dependencies(expression):
    dependencies = []  # Initialize an empty list to store dependencies
    for measure in df['Measure Name']:  # Loop through all measure names in the DataFrame
        if f'[{measure}]' in expression:  # Check if the measure name is in the expression
            dependencies.append(measure)  # If yes, add the measure name to the dependencies list
    return dependencies  # Return the list of dependencies

def build_directed_graph():
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        measure = row['Measure Name']  # Get the measure name
        G.add_node(measure)  # Add the measure as a node in the graph
        dependencies = extract_dependencies(row['Measure Expression'])  # Extract dependencies for the measure
        for dep in dependencies:
            G.add_edge(dep, measure)  # Add an edge from dependency to measure

# Function to find and graph all dependencies for a given measure
def graph_dependencies(measure_name):
    ancestors = nx.ancestors(G, measure_name)  # Find all upstream dependencies (ancestors)
    descendants = nx.descendants(G, measure_name)  # Find all downstream dependencies (descendants)
    subgraph_nodes = ancestors.union(descendants).union({measure_name})  # Include the measure itself
    subgraph = G.subgraph(subgraph_nodes)  # Create a subgraph with the dependencies and the measure
    
    '''
    size="30,30!": Sets the overall size of the graph in inches. The ! ensures the aspect ratio is maintained.
    dpi="300": Sets the dots per inch, determining the resolution of the image. Higher DPI means better image quality.
    ranksep="4": Sets the vertical space between ranks (levels) of nodes. Increasing this value spaces out the levels more.
    nodesep="1.5": Sets the horizontal space between nodes. Increasing this value spaces out the nodes horizontally.

    margin="0.8": Sets the margin around each node. This makes the node and its label less cramped.
    shape="box": Defines the shape of the nodes. Here, nodes will be displayed as boxes.
    fontsize="30": Sets the font size of the node labels, making them larger and more readable.
    width="2": Sets the width of each node box in inches.
    height="1": Sets the height of each node box in inches.
    '''

    
    # Dynamically adjust graph size and spacing based on the subgraph size
    num_nodes = len(subgraph.nodes)
    num_edges = len(subgraph.edges)
    size = (num_nodes * 3, num_edges * 2)  # Example formula for size
    ranksep = max(1, num_nodes / 5)  # Example formula for ranksep
    nodesep = max(0.5, num_nodes / 10)  # Example formula for nodesep

    # Convert the subgraph to a pygraphviz AGraph
    A = to_agraph(subgraph)

    # Set graph attributes dynamically
    A.graph_attr.update(size=f"{size[0]},{size[1]}!", dpi="300", ranksep=str(ranksep), nodesep=str(nodesep))
    A.node_attr.update(margin="0.8", shape="box", fontsize="30", width="2", height="1")

    # Highlight the selected measure
    selected_node = A.get_node(measure_name)
    selected_node.attr['color'] = 'red'
    selected_node.attr['style'] = 'filled'
    selected_node.attr['fillcolor'] = 'yellow'

    # Layout with dot (graphviz)
    A.layout(prog='dot')

    # Define the output path and save the graph as a PNG file
    output_path = f'/lakehouse/default/Files/{measure_name}_dependencies.png'
    A.draw(output_path, format='png')
    display(Image(filename=output_path))


In [ ]:
build_directed_graph()
graph_dependencies(measure)